#### Problem Statement:To create a reliable model that can effectively identify the credit amount to be given to each customer based on various factors, enabling the bank to make data-driven decisions for credit lending and risk management.

## Importing all Dependencies


In [50]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns",None)
pd.set_option('display.max_rows',None)

In [51]:
# prompt: write a code to mount drive

from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Reading the data

In [52]:
train=pd.read_csv(r'/content/drive/MyDrive/Resume projects/Train.csv',index_col=0)
train.head()

,account_info,duration_month,credit_history,purpose,credit_amount,savings_account,employment_st,poi,personal_status,gurantors,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
serial number,,,,,,,,,,,,,,,,,,,,
1,<0,6.0,A34,radio/TV,1169,unknown,A75,4,A93,A101,4.0,A121,67.0,A143,A152,2,A173,1,yes,foreigner
2,0-199,48.0,A32,radio/TV,5951,<100,A73,2,A92,A101,2.0,A121,22.0,A143,A152,1,A173,1,no,foreigner
3,unknown,12.0,A34,education,2096,<100,A74,2,A93,A101,3.0,A121,49.0,A143,A152,1,A172,2,no,foreigner
4,<0,42.0,A32,furniture/equipment,7882,<100,A74,2,A93,A103,4.0,A122,45.0,A143,A153,1,A173,2,no,foreigner
5,<0,24.0,A33,new_car,4870,<100,A73,3,A93,A101,4.0,A124,53.0,A143,A153,2,A173,2,no,foreigner


In [53]:
# Get the column names
cols = train.columns.tolist()

# Move 'Credit amount' to the end
cols.remove('credit_amount')
cols.append('credit_amount')

# Reorder the DataFrame
train = train[cols]
train.head()


,account_info,duration_month,credit_history,purpose,savings_account,employment_st,poi,personal_status,gurantors,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner,credit_amount
serial number,,,,,,,,,,,,,,,,,,,,
1,<0,6.0,A34,radio/TV,unknown,A75,4,A93,A101,4.0,A121,67.0,A143,A152,2,A173,1,yes,foreigner,1169
2,0-199,48.0,A32,radio/TV,<100,A73,2,A92,A101,2.0,A121,22.0,A143,A152,1,A173,1,no,foreigner,5951
3,unknown,12.0,A34,education,<100,A74,2,A93,A101,3.0,A121,49.0,A143,A152,1,A172,2,no,foreigner,2096
4,<0,42.0,A32,furniture/equipment,<100,A74,2,A93,A103,4.0,A122,45.0,A143,A153,1,A173,2,no,foreigner,7882
5,<0,24.0,A33,new_car,<100,A73,3,A93,A101,4.0,A124,53.0,A143,A153,2,A173,2,no,foreigner,4870


In [54]:
train.shape

(800, 20)

#### Descriptive Statistics Summary

In [55]:
train.describe()

,duration_month,poi,resident_since,age,credits_no,liables,credit_amount
count,772.000000,800.000000,787.000000,767.000000,800.000000,800.000000,800.000000
mean,20.645078,2.966250,2.822109,35.423729,1.396250,1.147500,3190.976250
std,12.374860,1.128806,1.105788,11.714471,0.569773,0.354825,2732.671789
min,4.000000,1.000000,1.000000,19.000000,1.000000,1.000000,250.000000
25%,12.000000,2.000000,2.000000,26.000000,1.000000,1.000000,1342.250000
50%,18.000000,3.000000,3.000000,32.000000,1.000000,1.000000,2281.500000
75%,24.000000,4.000000,4.000000,42.000000,2.000000,1.000000,3914.250000
max,72.000000,4.000000,4.000000,75.000000,4.000000,2.000000,15945.000000


In [56]:
test=pd.read_csv(r"/content/drive/MyDrive/Resume projects/Test.csv",index_col=0)
test.head()

,account_info,duration_month,credit_history,purpose,savings_account,employment_st,poi,personal_status,gurantors,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
serial number,,,,,,,,,,,,,,,,,,,
1,unknown,24.0,A34,education,<100,A75,4,A93,A101,4.0,A124,54.0,A143,A153,2,A173,2,no,foreigner
2,0-199,18.0,A34,radio/TV,<100,A75,3,A92,A103,4.0,A121,48.0,A141,A151,2,A172,1,yes,foreigner
3,<0,NaN,A34,furniture/equipment,<100,A75,1,A92,A101,4.0,A122,24.0,A143,A152,2,A173,1,no,foreigner
4,unknown,12.0,A34,radio/TV,unknown,A75,4,A93,A101,NaN,A123,35.0,A143,A152,2,A173,1,no,foreigner
5,0-199,12.0,A32,new_car,unknown,A71,1,A92,A101,2.0,A121,24.0,A143,A151,1,A171,1,no,foreigner


In [57]:
test.shape

(200, 19)

#### Handling missing Values

In [58]:
train.isna().sum()

,0
account_info,0
duration_month,28
credit_history,0
purpose,0
savings_account,0
employment_st,0
poi,0
personal_status,0
gurantors,0
resident_since,13


In [59]:
test.isna().sum()

,0
account_info,0
duration_month,10
credit_history,0
purpose,0
savings_account,0
employment_st,0
poi,0
personal_status,0
gurantors,0
resident_since,6


In [60]:
train.dtypes

,0
account_info,object
duration_month,float64
credit_history,object
purpose,object
savings_account,object
employment_st,object
poi,int64
personal_status,object
gurantors,object
resident_since,float64


In [61]:
for i in train.columns:
    print({i:train[i].unique()})  # to see the special characters

{'account_info': array(['<0', '0-199', 'unknown', '?', '>=200'], dtype=object)}
{'duration_month': array([ 6., 48., 12., 42., 24., 36., 30., 15.,  9., 10.,  7., 60., 18.,
       45., 11., 27.,  8., 54., nan, 14., 33., 16.,  4., 47., 13., 22.,
       39., 28.,  5., 26., 72., 40.])}
{'credit_history': array(['A34', 'A32', 'A33', 'A30', 'A31'], dtype=object)}
{'purpose': array(['radio/TV', 'education', 'furniture/equipment', 'new_car',
       'old_car', 'business', 'domestic_appliances', 'repairing', 'other',
       'upskilling'], dtype=object)}
{'savings_account': array(['unknown', '<100', '500-999', '>=1000', '100-499'], dtype=object)}
{'employment_st': array(['A75', 'A73', 'A74', 'A71', 'A72'], dtype=object)}
{'poi': array([4, 2, 3, 1])}
{'personal_status': array(['A93', 'A92', 'A91', 'A94'], dtype=object)}
{'gurantors': array(['A101', 'A103', '?', 'A102'], dtype=object)}
{'resident_since': array([ 4.,  2.,  3., nan,  1.])}
{'property_type': array(['A121', 'A122', 'A124', 'A123'], dtyp

In [62]:
train.replace('?',np.nan,inplace=True)

In [63]:
for i in test.columns:
    print({i:test[i].unique()})  # to see the special characters

{'account_info': array(['unknown', '0-199', '<0', '>=200', '?'], dtype=object)}
{'duration_month': array([24., 18., nan, 12., 36.,  6., 42., 15.,  8., 48., 45.,  4.,  9.,
       10., 27., 11., 30., 28., 16., 60., 13.])}
{'credit_history': array(['A34', 'A32', 'A31', 'A30', 'A33'], dtype=object)}
{'purpose': array(['education', 'radio/TV', 'furniture/equipment', 'new_car',
       'old_car', 'business', 'domestic_appliances', 'other', 'repairing',
       'upskilling'], dtype=object)}
{'savings_account': array(['<100', 'unknown', '500-999', '100-499', '>=1000'], dtype=object)}
{'employment_st': array(['A75', 'A71', 'A74', 'A72', 'A73'], dtype=object)}
{'poi': array([4, 3, 1, 2])}
{'personal_status': array(['A93', 'A92', 'A94', 'A91'], dtype=object)}
{'gurantors': array(['A101', 'A103', '?', 'A102'], dtype=object)}
{'resident_since': array([ 4., nan,  2.,  1.,  3.])}
{'property_type': array(['A124', 'A121', 'A122', 'A123'], dtype=object)}
{'age': array([54., 48., 24., 35., 26., 65., 55., 2

In [64]:
test.replace('?',np.nan,inplace=True)

#### Filling missing values

In [65]:
for x in train.columns:
    if train[x].dtype=='object' or train[x].dtype=='bool':
        train[x].fillna(train[x].mode()[0],inplace=True)
    elif train[x].dtype=='int64' or train[x].dtype=='float64':
        train[x].fillna(round(train[x].mean()),inplace=True)

In [66]:
for x in test.columns:
    if test[x].dtype=='object' or test[x].dtype=='bool':
        test[x].fillna(test[x].mode()[0],inplace=True)
    elif test[x].dtype=='int64' or test[x].dtype=='float64':
        test[x].fillna(round(test[x].mean()),inplace=True)

In [67]:
print(train.isna().sum())
print(test.isna().sum())

account_info        0
duration_month      0
credit_history      0
purpose             0
savings_account     0
employment_st       0
poi                 0
personal_status     0
gurantors           0
resident_since      0
property_type       0
age                 0
installment_type    0
housing_type        0
credits_no          0
job_type            0
liables             0
telephone           0
foreigner           0
credit_amount       0
dtype: int64
account_info        0
duration_month      0
credit_history      0
purpose             0
savings_account     0
employment_st       0
poi                 0
personal_status     0
gurantors           0
resident_since      0
property_type       0
age                 0
installment_type    0
housing_type        0
credits_no          0
job_type            0
liables             0
telephone           0
foreigner           0
dtype: int64


#### converting categorical variables to numerical

In [68]:
colname=[]
for x in train.columns:
    if train[x].dtype=='object':
        colname.append(x)
print(colname)

['account_info', 'credit_history', 'purpose', 'savings_account', 'employment_st', 'personal_status', 'gurantors', 'property_type', 'installment_type', 'housing_type', 'job_type', 'telephone', 'foreigner']


In [69]:
# For preprocessing the data
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for x in colname:
    train[x]=le.fit_transform(train[x])

In [70]:
colname=[]
for x in test.columns:
    if test[x].dtype=='object':
        colname.append(x)
print(colname)

['account_info', 'credit_history', 'purpose', 'savings_account', 'employment_st', 'personal_status', 'gurantors', 'property_type', 'installment_type', 'housing_type', 'job_type', 'telephone', 'foreigner']


In [71]:
# For preprocessing the data
from sklearn.preprocessing import LabelEncoder

le=LabelEncoder()

for x in colname:
    test[x]=le.fit_transform(test[x])

In [72]:
test.head()

,account_info,duration_month,credit_history,purpose,savings_account,employment_st,poi,personal_status,gurantors,resident_since,property_type,age,installment_type,housing_type,credits_no,job_type,liables,telephone,foreigner
serial number,,,,,,,,,,,,,,,,,,,
1,3,24.0,4,2,2,4,4,2,0,4.0,3,54.0,2,2,2,2,2,0,0
2,0,18.0,4,7,2,4,3,1,2,4.0,0,48.0,0,0,2,1,1,1,0
3,1,22.0,4,3,2,4,1,1,0,4.0,1,24.0,2,1,2,2,1,0,0
4,3,12.0,4,7,4,4,4,2,0,3.0,2,35.0,2,1,2,2,1,0,0
5,0,12.0,2,4,4,0,1,1,0,2.0,0,24.0,2,0,1,0,1,0,0


#### Creating X and Y

In [73]:
X=train.values[:,0:-1]
Y=train.values[:,-1]
Y=Y.astype(int)

In [74]:
print(X)

[[ 1.  6.  4. ...  1.  1.  0.]
 [ 0. 48.  2. ...  1.  0.  0.]
 [ 3. 12.  4. ...  2.  0.  0.]
 ...
 [ 3. 12.  4. ...  1.  0.  0.]
 [ 3. 24.  3. ...  1.  1.  0.]
 [ 0.  9.  2. ...  1.  0.  0.]]


In [75]:
from sklearn.model_selection import train_test_split

#Split the data into test and train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2,
                                                    random_state=10)

In [76]:
print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(640, 19)
(160, 19)
(640,)
(160,)


#### Random Forest Regressor

In [77]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Create the base model object with fixed parameters
model_EXT = RandomForestRegressor(random_state=10, bootstrap=True)

# Define the hyperparameters and their values for grid search
parameter_space = {
    'criterion': ['mse', 'friedman_mse', 'mae'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Perform grid search with cross-validation
clf = GridSearchCV(model_EXT, parameter_space, n_jobs=-1, cv=5)
clf.fit(X_train, Y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(random_state=10), n_jobs=-1,
             param_grid={'criterion': ['mse', 'friedman_mse', 'mae'],
                         'max_depth': [None, 10, 20, 30],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]})

In [78]:
# Get the best hyperparameters
best_params = clf.best_params_
best_params


{'criterion': 'friedman_mse',
 'max_depth': 10,
 'min_samples_leaf': 2,
 'min_samples_split': 10}

In [79]:
#predicting using the Random_Forest_Regressor
from sklearn.ensemble import RandomForestRegressor

model_RandomForest=RandomForestRegressor(n_estimators=500,criterion='squared_error',
 max_depth=20,
 min_samples_leaf= 5,
 min_samples_split=10,
 random_state=10, bootstrap=True,n_jobs=-1)

#fit the model on the data and predict the values
model_RandomForest.fit(X_train,Y_train)

Y_pred=model_RandomForest.predict(X_test)

In [80]:
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2=r2_score(Y_test,Y_pred)
print("R-squared:",r2)

rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print("RMSE:",rmse)

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)  # formula of adj R2 shape(no of ind vars)
print("Adj R-square:",adjusted_r_squared)

R-squared: 0.6609293878956712
RMSE: 1610.7041259512782
Adj R-square: 0.6526699755495402


In [81]:
# Define the normalization factor
normalization_factor = np.max(Y_test) - np.min(Y_test)

# Calculate the normalized RMSE
normalized_rmse = (1 - rmse / normalization_factor) * 100

# Print the normalized RMSE score
print("Normalized RMSE Score:", normalized_rmse)

Normalized RMSE Score: 88.84785622134406


## Extreme Gradient Boosting

In [82]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming X_train, y_train, X_test, and y_test are your training and testing data
# Convert data to DMatrix format required by XGBoost
dtrain = xgb.DMatrix(X_train, label=Y_train)
dtest = xgb.DMatrix(X_test, label=Y_test)

# Define parameters for XGBoost
params = {
    'objective': 'reg:squarederror',  # Specify the objective as regression
    'eval_metric': 'rmse'  # Evaluation metric is Root Mean Squared Error
}

# Train the model
num_rounds = 100 # Number of boosting rounds
xgb_model = xgb.train(params, dtrain, num_rounds)

# Make predictions on the test data
Y_pred = xgb_model.predict(dtest)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(Y_test, Y_pred))

# Print the RMSE score
print("RMSE Score:", rmse)


RMSE Score: 1829.2422750219719


In [83]:
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np
 # ytest is used to cross check
r2=r2_score(Y_test,Y_pred) # unseen data
print("R-squared:",r2)

rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))  # manually applied sqrt
print("RMSE:",rmse)

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)  # formula of adj R2 shape(no of ind vars)
print("Adj R-square:",adjusted_r_squared)

R-squared: 0.5626782601179681
RMSE: 1829.2422750219719
Adj R-square: 0.5520255510695595


#### Gradient boosting

In [84]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

# Assuming X_train, y_train, X_test, and y_test are your training and testing data
# Create and train the Gradient Boosting Regressor model
gbm_regressor = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3,max_features=11,random_state=42)
# You can adjust n_estimators, learning_rate, max_depth, and other parameters as needed
gbm_regressor.fit(X_train, Y_train)

# Make predictions
Y_pred= gbm_regressor.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(Y_test,Y_pred))

# Print the RMSE score
print("RMSE Score:", rmse)

RMSE Score: 1627.4166743147575


In [85]:
# prompt: now write a code to use the gbm model to predict credit amount for test data the file which is provided above

# Make predictions on the test data
test_predictions = gbm_regressor.predict(test.values)

# Add predictions as a new column to the test DataFrame
test['predicted_credit_amount'] = test_predictions

# Display the test DataFrame with predictions
print(test.head())


               account_info  duration_month  credit_history  purpose  \
serial number                                                          
1                         3            24.0               4        2   
2                         0            18.0               4        7   
3                         1            22.0               4        3   
4                         3            12.0               4        7   
5                         0            12.0               2        4   

               savings_account  employment_st  poi  personal_status  \
serial number                                                         
1                            2              4    4                2   
2                            2              4    3                1   
3                            2              4    1                1   
4                            4              4    4                2   
5                            4              0    1                1  